# Phase 1 :- Installation

In [1]:

print("⏳ Installing Google Agent Development Kit (ADK)...")
!pip install -qU google-adk

print("⏳ Installing Google Gen AI SDK (The Brain)...")
!pip install -qU google-genai

print("⏳ Installing Financial & Search Tools (The Senses)...")
!pip install -qU yfinance pandas_ta duckduckgo-search

print("✅ PHASE 1 COMPLETE: Environment is ready.")

⏳ Installing Google Agent Development Kit (ADK)...
⏳ Installing Google Gen AI SDK (The Brain)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 10.6 MB/s eta 0:00:00
⏳ Installing Financial & Search Tools (The Senses)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 139.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 148.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

# Phase 2: The Senses (Setup & Brain)

In [2]:
# [CELL 2] - PHASE 2: SETUP & BRAIN INITIALIZATION
import os
import getpass
from google.genai import Client # Imported here to fix NameError later

# 1. SECURE API KEY ENTRY
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑 Enter your Gemini API Key: ")

# 2. CONFIGURE ENVIRONMENT
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
MODEL_NAME = "gemini-2.0-flash-exp"

# 3. INITIALIZE CLIENT (THE FIX)
# We define 'client' here globally so all agents can find it.
client = Client(api_key=os.environ["GOOGLE_API_KEY"])

print(f"✅ SYSTEM READY.")
print(f"   Target Model: {MODEL_NAME}")
print(f"   Client Connection: Established")

✅ SYSTEM READY.
   Target Model: gemini-2.0-flash-exp
   Client Connection: Established


# Phase 2 (Continued): Building MCP Tools

In [3]:
# PHASE 2: DEFINING THE MCP TOOLS (SENSES)
import yfinance as yf

# --- TOOL 1: THE FINANCIAL EYE (Price Data) ---
def get_stock_price(symbol: str) -> str:
    """
    Fetches the current stock price and daily change.
    Args:
        symbol: Ticker symbol (e.g., AAPL)
    """
    try:
        ticker = yf.Ticker(symbol)
        history = ticker.history(period="1d")
        
        if history.empty:
            return f"Error: No price data for {symbol}."
            
        current_price = history['Close'].iloc[-1]
        open_price = history['Open'].iloc[-1]
        change_pct = ((current_price - open_price) / open_price) * 100
        
        return f"{symbol.upper()} Price: ${current_price:.2f} | Change: {change_pct:.2f}%"
    except Exception as e:
        return f"Error fetching price: {str(e)}"

# --- TOOL 2: THE NEWS READER (Updated to use YFinance) ---
def get_market_news(symbol: str) -> str:
    """
    Fetches the latest financial news for a company directly from Yahoo Finance.
    Args:
        symbol: Ticker symbol (e.g., AAPL)
    """
    try:
        ticker = yf.Ticker(symbol)
        news_list = ticker.news
        
        if not news_list:
            return f"No news found for {symbol}."
            
        # Format the top 3 articles
        summary = f"--- LATEST NEWS FOR {symbol} ---\n"
        for i, article in enumerate(news_list[:3]):
            title = article.get('title', 'No Title')
            link = article.get('link', 'No Link')
            summary += f"{i+1}. {title} ({link})\n"
            
        return summary
    except Exception as e:
        return f"Error fetching news: {str(e)}"

print("✅ PHASE 2 COMPLETE: Tools are built.")

✅ PHASE 2 COMPLETE: Tools are built.


# Phase 3: The Workforce (The Specialist Agents)

In [4]:
# PHASE 3: AGENTS WITH SIMULATION FALLBACK
from google.genai.types import Tool, FunctionDeclaration, Schema, Type, GenerateContentConfig
from google.genai.errors import ClientError
import time

# --- 1. TOOLS (Unchanged) ---
price_tool = Tool(
    function_declarations=[
        FunctionDeclaration(
            name="get_stock_price",
            description="Fetch current stock price and daily percentage change.",
            parameters=Schema(
                type=Type.OBJECT,
                properties={
                    "symbol": Schema(type=Type.STRING, description="The stock ticker symbol (e.g., 'AAPL')")
                },
                required=["symbol"]
            )
        )
    ]
)

news_tool = Tool(
    function_declarations=[
        FunctionDeclaration(
            name="get_market_news",
            description="Fetch top 3 latest financial news headlines.",
            parameters=Schema(
                type=Type.OBJECT,
                properties={
                    "symbol": Schema(type=Type.STRING, description="The stock ticker symbol (e.g., 'AAPL')")
                },
                required=["symbol"]
            )
        )
    ]
)

# --- 2. AGENT CLASS (INDESTRUCTIBLE VERISON) ---
class Agent:
    def __init__(self, name, model, tools, instructions):
        self.name = name
        self.model = model
        self.tools = tools
        self.instructions = instructions

    def work(self, prompt):
        print(f"\n🤖 {self.name} is working...")
        
        my_config = None
        if self.tools:
            my_config = GenerateContentConfig(tools=self.tools)

        # Retry Loop
        max_retries = 2 # Reduced retries to save time
        for attempt in range(max_retries):
            try:
                response = client.models.generate_content(
                    model=self.model,
                    contents=f"{self.instructions}\n\nTASK: {prompt}",
                    config=my_config 
                )
                return response # Success!
            
            except Exception as e:
                print(f"   ⚠️ API BUSY. Pausing 10s...")
                time.sleep(10)
        
        # --- FALLBACK MODE ---
        # If API fails 3 times, return a MOCK OBJECT instead of crashing.
        print(f"   ⚠️ {self.name} switched to SIMULATION MODE (Quota Hit).")
        
        class MockResponse:
            def __init__(self, text): self.text = text
            
        if "Technical" in self.name:
            return MockResponse("SIMULATION DATA: RSI is 35 (Oversold). Price is trending up above 50 MA. Signal: BULLISH.")
        else:
            return MockResponse("SIMULATION DATA: News sentiment is positive. CEO announced new product. Signal: POSITIVE.")

# --- 3. RECRUIT THE TEAM ---
# We stick to the model we know exists, but now we have a safety net.
MODEL_NAME = "gemini-2.0-flash-exp"

technical_agent = Agent(
    name="Technical Analyst",
    model=MODEL_NAME, 
    tools=[price_tool], 
    instructions="You are a Technical Analyst. Analyze price movements. Use 'get_stock_price'. Ignore news."
)

news_agent = Agent(
    name="News Analyst",
    model=MODEL_NAME, 
    tools=[news_tool], 
    instructions="You are a Financial Journalist. Analyze market sentiment. Use 'get_market_news'. Ignore math."
)

print(f"✅ PHASE 3 COMPLETE: Agents are now Crash-Proof.")

✅ PHASE 3 COMPLETE: Agents are now Crash-Proof.


# Phase 4 :- THE SUPERVISOR (The Boardroom)

In [5]:
# PHASE 4: THE SUPERVISOR (With Simulation Fallback)
import textwrap
import time

class SupervisorAgent:
    def __init__(self, name, model):
        self.name = name
        self.model = model
    
    def make_decision(self, stock_symbol):
        print(f"\n👔 {self.name} is calling a meeting about {stock_symbol}...")
        
        # 1. Get Reports (Agents handle their own fallbacks now)
        tech_report = technical_agent.work(f"Analyze {stock_symbol}")
        print(f"   📊 Quant says: {tech_report.text[:50]}...") 
        
        news_report = news_agent.work(f"Check news for {stock_symbol}")
        print(f"   📰 Journalist says: {news_report.text[:50]}...") 
        
        # 2. Decision
        print("   ...Supervisor is Thinking...")
        final_prompt = f"""
        Head Portfolio Manager Decision.
        REPORT A: {tech_report.text}
        REPORT B: {news_report.text}
        DECISION (Buy/Sell/Hold) & REASONING:
        """
        
        try:
            decision = client.models.generate_content(
                model=self.model,
                contents=final_prompt
            )
            return decision.text
        except Exception as e:
            # SAFETY NET: If Supervisor fails, force a decision so execution finishes.
            print("   ⚠️ SUPERVISOR QUOTA HIT. Using Simulation Decision.")
            return f"""
            Subject: 🚨 SIMULATION ALERT: BUY {stock_symbol}
            
            Decision: BUY (Simulated)
            Reasoning: API Quota limit reached. This is a fallback decision to demonstrate the email system.
            Technical data was bullish and News was positive.
            """

supervisor = SupervisorAgent(
    name="Chief Investment Officer",
    model="gemini-2.0-flash-exp"
)

print("✅ PHASE 4 COMPLETE: Supervisor is Crash-Proof.")

✅ PHASE 4 COMPLETE: Supervisor is Crash-Proof.


# Phase 5:- Output & Execution

In [6]:
# PHASE 5 : EXECUTION & NOTIFICATION
import time

# --- STEP 1: THE EMAILER (The Action Tool) ---
def send_email_alert(symbol, content):
    """
    Simulates sending an HTML email via SMTP.
    """
    print("\n" + "!"*60)
    print(f"📧 URGENT EMAIL SENT TO SUBSCRIBERS: {symbol.upper()}")
    print("!"*60)
    print(f"Subject: 🚨 Trade Alert for {symbol.upper()}")
    print("-" * 30)
    print(content)
    print("-" * 30)
    print("✅ Email dispatched successfully.")

# --- THE MAIN WORKFLOW ---
def run_hedge_fund_simulation(ticker_symbol):
    print(f"\n🚀 STARTING SIMULATION FOR: {ticker_symbol.upper()}")
    print("="*60)
    
    start_time = time.time()
    
    # 1. THE BOARDROOM (Phase 4 Logic)
    decision_text = supervisor.make_decision(ticker_symbol)
    
    # 2. THE TRIGGER (Phase 5 Logic)
    print("\n⚙️ EVALUATING DECISION FOR ACTION...")
    
    # We check if the keywords exist in the decision text
    if "BUY" in decision_text.upper() or "SELL" in decision_text.upper():
        send_email_alert(ticker_symbol, decision_text)
    else:
        print(f"ℹ️ Decision is HOLD/NEUTRAL. No email sent to avoid spam.")
    
    # 3. Performance Stats
    duration = time.time() - start_time
    print(f"\n✅ CYCLE COMPLETE. Time taken: {duration:.2f} seconds.")

# --- EXECUTE THE AGENT ---
target_stock = "NVDA" 
run_hedge_fund_simulation(target_stock)


🚀 STARTING SIMULATION FOR: NVDA

👔 Chief Investment Officer is calling a meeting about NVDA...

🤖 Technical Analyst is working...
   ⚠️ API BUSY. Pausing 10s...
   ⚠️ API BUSY. Pausing 10s...
   ⚠️ Technical Analyst switched to SIMULATION MODE (Quota Hit).
   📊 Quant says: SIMULATION DATA: RSI is 35 (Oversold). Price is tr...

🤖 News Analyst is working...
   ⚠️ API BUSY. Pausing 10s...
   ⚠️ API BUSY. Pausing 10s...
   ⚠️ News Analyst switched to SIMULATION MODE (Quota Hit).
   📰 Journalist says: SIMULATION DATA: News sentiment is positive. CEO a...
   ...Supervisor is Thinking...
   ⚠️ SUPERVISOR QUOTA HIT. Using Simulation Decision.

⚙️ EVALUATING DECISION FOR ACTION...

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
📧 URGENT EMAIL SENT TO SUBSCRIBERS: NVDA
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Subject: 🚨 Trade Alert for NVDA
------------------------------

            Subject: 🚨 SIMULATION ALERT: BUY NVDA
            
            Decision: BUY (